In [107]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import random
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [91]:
WVS_df = pd.read_csv("/Users/vishwakirti/Desktop/Assignments Fall 2019/IMT 574/wvs.csv",sep='\t', index_col = False)
WVS_df.shape
WVS_df.describe()
WVS_df.head(5)
#WVS_df.shape

# Load the data. How many responses and variables do we have?
# We have 90350 responses and 328 variables




,V2,V4,V5,V6,V7,V8,V9,V10,V11,V12,...,MN_228S8,MN_229A,MN_230A,MN_233A,MN_237B1,MN_249A1,MN_249A3,I_RELIGBEL,I_NORM1,I_VOICE1
0,12,1,1,1,-2,1,1,2,1,1,...,3,-3,-3,-3,-3,1,1,0.0,1.0,0.00
1,12,1,2,3,4,2,2,2,2,2,...,3,-3,-3,-3,-3,2,-1,0.0,1.0,0.66
2,12,1,3,2,4,2,1,2,2,2,...,4,1,1,2,-3,1,1,0.0,1.0,0.33
3,12,1,1,3,4,3,1,2,1,2,...,2,2,1,2,-3,1,2,0.0,1.0,0.00
4,12,1,1,1,2,1,1,1,3,2,...,2,2,1,2,-3,1,2,0.0,1.0,0.66


In [92]:
# Create a summary table over all responses for V204: is abortion justifable. 
# How many non- missing responses (i.e. positive answers) do you find? Describe the the opinion about the 
# abortion among the global pool of respondents.

# There are a total of 85742 "no missing responses"

WVS_df_filter = WVS_df[['V204']]
WVS_df_filter
print(WVS_df_filter.shape)
WVS_df_V204 = WVS_df_filter[WVS_df_filter.V204 > 0]
WVS_df_V204.shape
WVS_df_V204

WVS_df_V204_max = WVS_df_filter[WVS_df_filter.V204 > 6]
print(WVS_df_V204_max.shape)

# Summary table
pd.DataFrame(pd.merge(WVS_df_V204.mean().reset_index(name='Mean'), WVS_df_V204.std().reset_index(name='Std_dev'),on=['index'])).sort_values(by ='index')

# As per the summary table we can see that the mean of responses is 3.22 which means most of the global population thinks 
# abortion is not justifiable. 
# I also tried to find how many respondednts consider Abortion justifiable i.e. response between 7 to 10.
# The result showed that only 12853 i.e 13% of the total think it is justifiable.

(90350, 1)
(12853, 1)


,index,Mean,Std_dev
0,V204,3.225024,2.764319


In [93]:
# Now remove missings. We do it in two ways:
#    (a) remove everything that are not positive integers for V204 and V2 (country).
#    (b) for all other variables, remove the missings in the sense of missing value on computer. 
#        You may leave negative answers in the data, otherwise I am afraid your sample size collapses.
#What is the final number of observations?

#WVS_df_filter1 = WVS_df[['V2','V204']]
#del WVS_df_filter1['index']

WVS_df_V2_V204_1 = WVS_df[WVS_df.V204 > 0]
WVS_df_V2_V204 = WVS_df_V2_V204_1[WVS_df_V2_V204_1.V2 > 0]
print(WVS_df_V2_V204.shape)

#WVS_df_V2_V204.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
#WVS_df_V2_V204 = WVS_df_V2_V204.dropna()
#print(WVS_df_V2_V204.shape)

Header  = WVS_df_V2_V204.columns
print(Header)


Header = Header.drop(['V2','V204'])
print(Header)
WVS_df_V2_V204.dropna(axis=0, subset=Header, inplace=True)
print(WVS_df_V2_V204.shape)
WVS_df_V2_V204
# The final number of observation is 79267

(85742, 328)
Index(['V2', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12',
       ...
       'MN_228S8', 'MN_229A', 'MN_230A', 'MN_233A', 'MN_237B1', 'MN_249A1',
       'MN_249A3', 'I_RELIGBEL', 'I_NORM1', 'I_VOICE1'],
      dtype='object', length=328)
Index(['V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13',
       ...
       'MN_228S8', 'MN_229A', 'MN_230A', 'MN_233A', 'MN_237B1', 'MN_249A1',
       'MN_249A3', 'I_RELIGBEL', 'I_NORM1', 'I_VOICE1'],
      dtype='object', length=326)
(79267, 328)


,V2,V4,V5,V6,V7,V8,V9,V10,V11,V12,...,MN_228S8,MN_229A,MN_230A,MN_233A,MN_237B1,MN_249A1,MN_249A3,I_RELIGBEL,I_NORM1,I_VOICE1
0,12,1,1,1,-2,1,1,2,1,1,...,3,-3,-3,-3,-3,1,1,0.0,1.0,0.00
1,12,1,2,3,4,2,2,2,2,2,...,3,-3,-3,-3,-3,2,-1,0.0,1.0,0.66
2,12,1,3,2,4,2,1,2,2,2,...,4,1,1,2,-3,1,1,0.0,1.0,0.33
3,12,1,1,3,4,3,1,2,1,2,...,2,2,1,2,-3,1,2,0.0,1.0,0.00
4,12,1,1,1,2,1,1,1,3,2,...,2,2,1,2,-3,1,2,0.0,1.0,0.66
5,12,1,2,2,2,4,1,2,1,2,...,3,2,1,1,-3,1,2,0.0,1.0,0.00
6,12,1,1,1,1,1,1,2,2,1,...,3,2,2,2,-3,1,1,0.0,1.0,0.66
7,12,1,1,1,1,2,2,2,1,2,...,3,1,1,2,-3,2,2,0.0,1.0,0.00
8,12,1,1,1,2,2,2,2,2,2,...,3,2,1,1,-3,-3,-3,0.0,1.0,0.33
9,12,1,1,1,2,1,1,1,1,2,...,3,-3,-3,-3,0,-3,-3,0.0,1.0,0.66


In [94]:

# (2pt) In order to simplify the analysis below, create a new binary variable abortion as



WVS_df_V2_V204.loc[WVS_df_V2_V204.V204 > 3, 'abortion'] = 0
WVS_df_V2_V204.loc[WVS_df_V2_V204.V204 <= 3, 'abortion'] = 1
WVS_df_V2_V204

WVS_pearson = WVS_df_V2_V204.corr(method='pearson')

#sorting using coorelation with abortion
WVS_pearson.sort_values(by=['abortion'])
WVS_pearson.reset_index(inplace = True)

#dropping all columns except index and abortion
WVS_matrix = WVS_pearson[['index','abortion']]
WVS_matrix

# The table represents the correlation of abortion with all the other variables mentioned in the index.
# The highest correlation would be 1 which means with every change in he variable there would be an equivalent change
# in the abortion.


,index,abortion
0,V2,-0.038487
1,V4,-0.051921
2,V5,0.028951
3,V6,0.070634
4,V7,-0.003503
5,V8,-0.075076
6,V9,-0.314117
7,V10,-0.020985
8,V11,-0.025678
9,V12,0.077677


In [ ]:
WVS_df_country = WVS_df_V2_V204.rename(columns={"V2": "country"})
WVS_df_country.head(5)
WVS_df_country.shape

WVS_dummies = pd.get_dummies(WVS_df_country, columns = ['country'])
WVS_dummies.shape

#WVS_country_dum = [col for col in WVS_dummies if col.startswith('country')]
#WVS_country_dum.shape

WVS_country_dum = WVS_dummies.loc[:, WVS_dummies.columns.str.startswith('country')]
WVS_country_dum.shape

#WVS_dummies
# total number of rows = 79267 and total number of columns = 386

# there are a total of 58 country dummy columns
# hence removing those 58 we will have 328 columns 

#WVS_dummies_drop = WVS_dummies.drop(columns=['country'])
#WVS_dummies_drop.shape

In [102]:
# 2 Implement Cross-Validation (40pt) Now it's time to write your own code that does k-fold CV.


#selecting 8000 responses 

WVS_sample = WVS_dummies.sample(n=8000,random_state=1)

def CV(k, model, X, y,normal):
    sample_shuffle = WVS_sample.reindex(np.random.permutation(WVS_sample.index))
    if normal == 1:
        sc = MinMaxScaler() 
        sample_shuffle = sc.fit_transform(sample_shuffle)
        sample_shuffle = pd.DataFrame(sample_shuffle, columns = WVS_dummies.columns)
    
    f1score = []
    accuracy = []
    
    

    chk = [sample_shuffle[i::k] for i in range(k)]  
    for i in np.arange(k):
        Validation = chk[i]
        Train = sample_shuffle[~sample_shuffle.index.isin(Validation.index)]
        Validation_X = Validation[X]
        Train_X = Train[X]
        Validation_Y = np.asarray(Validation[y])
        Train_Y = np.asarray(Train[y])
        fit = model.fit(Train_X,Train_Y)
        pred = fit.predict(Validation_X)
        f1score.append(f1_score(Validation_Y,pred))
        accuracy.append(accuracy_score(Validation_Y,pred))
    return (np.mean(f1score),np.mean(accuracy))


In [111]:
# First, use k-NN and experiment with a few different k-s.
# Separate your training data into X (features), and y (target). Target will be the abortion variable, X are all the other features.
# (2pt) pick a k and set up the k-NN model. Use your freshly-minted CV routine to cross-validate accuracy and F-score of your k-NN model.
# (5pt) Try a few different k-NN models (pick different k, choose to normalize/not-to-normalize your features).
# (4pt) Present the results from your best k-NN model. 
# Note: as you are using two metrics here, you may end up with di􏰃erent models performing better according to 
# di􏰃erent measures.


X = set(WVS_dummies.columns)
X.remove('abortion')
y = WVS_dummies.columns.difference(X)



#Let us take Neighbours = 17 and  Normalization =0 
NearNeighbour1 = KNeighborsClassifier(n_neighbors=17)
print(CV(5,NearNeighbour1,X,y,0))

# The result was : (0.8546103491540975, 0.806)

#Let us take Neighbours = 17 and  Normalization =1
NearNeighbour2 = KNeighborsClassifier(n_neighbors=17)
print(CV(5,NearNeighbour2,X,y,1))

# The result was : 0.8494784410551744, 0.8002500000000001)

#Let us take Neighbours = 9 and  Normalization =0
NearNeighbour3 = KNeighborsClassifier(n_neighbors=9)
print(CV(5,NearNeighbour3,X,y,0))

# The result was : (0.8549829679587196, 0.8074999999999999)


#Let us take Neighbours = 9 and  Normalization =1
NearNeighbour4 = KNeighborsClassifier(n_neighbors=9)
print(CV(5,NearNeighbour4,X,y,1))

# The result was : (0.8471861134531056, 0.7983750000000001)


# The best F1 score (0.8549) and accuracy (0.807) we got from the kNN model with 9 neighbors and Normlization = 0.


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

(0.8546103491540975, 0.806)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

(0.8494784410551744, 0.8002500000000001)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for

(0.8549829679587196, 0.8074999999999999)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

(0.8471861134531056, 0.7983750000000001)


In [108]:
#3.2 Logistic Regression (9pt)

#Now repeat the process above with logistic regression. As we have a myriad of features anyway, we are not going 
#to do any feature engineering. Just a plain logistic regression.
LogisticReg = LogisticRegression()
CV(5,LogisticReg,X,y,0)

# The result was: (0.9995105009870615, 0.999375)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

(0.9995105009870615, 0.999375)

In [112]:
#.3 SVM (15pt) Now repeat the process with support vector machines while choosing between a few different kernels 
#and kernel options, such as degree for polynomial kernels. Hint: I have mixed experience with sklearn version of SVM.
#I recommend to limit the number of iterations, initially maybe to just 1000, in order to ensure your model actually 
#terminates.


SVC1=SVC(kernel='poly', degree=8,gamma=3)
print(CV(5,SVC1,X,y,0))
# The result was (0.9739278378184965, 0.966625)

SVC2=SVC(kernel='linear')
print(CV(5,SVC2,X,y,0))
# The result was (1.0, 1.0)

SVC3=SVC(kernel='poly', degree=3,gamma='auto')
print(CV(5,SVC3,X,y,0))
# The result was (0.9892183322923955, 0.9862500000000001)

SVC4=SVC(kernel='sigmoid', gamma=8)
print(CV(5,SVC4,X,y,0))
# The result was (0.7779344132865424, 0.6366250000000001)


# Only the sigmoid kernel had low accuracy and score.Recall actually calculates how many of the Actual Positives 
# our model capture through labeling it as Positive (True Positive). F1 Score is needed when you want to seek 
# a balance between Precision and Recall. Accuracy can be largely contributed by a large number of True Negatives. 
# F1 Score might be a better measure to use if we need to seek a balance between Precision and Recall and there 
# is an uneven class distribution (large number of Actual Negatives).


# (1pt) If you have to repeat the exercise with a single model (and you have, see below), which one will you pick?
# I would pick logistic regression or the Liner kernel SVM model because of theri accuracy and F1 score.
# However, SVM model took a lot of time to run compared to Logistic regression.

/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataC

(0.9739278378184965, 0.966625)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataC

(1.0, 1.0)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataC

(0.9892183322923955, 0.9862500000000001)


/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataC

(0.7779344132865424, 0.6366250000000001)


In [ ]:
# How large a role does country play? (20pt) Here we switch from machine learning to social sciences. 
# Public opinion differs from country to country, but also inside the countries. Does the fact that we include 
# country code in data help us to substantially improve the predictions? You pick the best ML method from above. 
# You estimate two sets of models: one with country information included, and one where it is removed. 
# Is the former noticeably better than the latter?

# (10pt) Pick your best ML method based you designed above. Cross-validate the accuracy of abortion variable 
# using all the features, including country dummies and report the accuracy. Essentially you repeat here what 
# you did above, so you can also just copy the result from above

In [113]:
LogisticReg = LogisticRegression()
CV(5,LogisticReg,X,y,0)

# The result was (0.9991178914796507, 0.998875)

/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

(0.9991178914796507, 0.998875)

In [119]:
# (15pt) Now remove all the country dummies, but keep the other variables intact. And repeat.

#Remove all Country Dummies
no_country_dums = WVS_dummies[WVS_dummies.columns.drop(list(WVS_dummies.filter(regex='country')))]

#Remove the country Dummy Varibales fromt the feature set.

no_country_dums_cols = set(no_country_dums.columns)
no_country_dums_cols.remove('abortion')
difference = no_country_dums.columns.difference(no_country_dums_cols)
#no_dummy_logistic = LogisticRegression()
print ("Logistic",CV(5,LogisticReg,no_country_dums_cols,difference,0))
no_country_dums.columns

# The result is Logistic (0.9995065971184511, 0.999375)
# we can see that the accuracy has slighly increased after removing the dummies.
# The activity did not help to increase the accuracy significantly but slightly.

/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/vishwakirti/Desktop/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

Logistic (0.9992146494926726, 0.999)


Index(['V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13',
       ...
       'MN_229A', 'MN_230A', 'MN_233A', 'MN_237B1', 'MN_249A1', 'MN_249A3',
       'I_RELIGBEL', 'I_NORM1', 'I_VOICE1', 'abortion'],
      dtype='object', length=328)